# week6_hw PCA and K-means

In [1]:
import jieba
import os
import pandas as pd
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfVectorizer
from datetime import timedelta, date
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA as PCA
#from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [2]:
import matplotlib.pyplot as plt
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

In [3]:
def is_chinese(uchar):         
    if u'\u4e00' <= uchar<=u'\u9fff':
        return True
    else:
        return False

In [4]:
# 專有名詞
jieba.load_userdict('ProperN.txt')

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/bf/kxrhqhkd3s1263kmtf84x8zw0000gn/T/jieba.cache
Loading model cost 0.715 seconds.
Prefix dict has been built succesfully.


In [5]:
folders = ["中電/", "金雨/", "大同/", "中興電/", "聲寶/", "東元/"]

In [6]:
imp_words = ["經營權",  "董事會",  "請辭", "接任", "出任", "兼任", "辭去", "入主", "推選", "另聘"]

In [7]:
def get_txts(folder):
    all_files = []
    for f in os.listdir("News/" + folder):
        if not f.startswith('.'):
            all_files.append(f)
    #os.listdir("News/" + folder)   # imagine you're one directory above test dir
    return all_files

In [8]:
# 讀取新聞檔案，並使用jieba切詞
def textMining(folder, file):
    with open("./News/" + folder + file, 'rt',encoding="utf-8", errors='ignore') as txt:
        data = txt.read()
    #text segmentation
    seg_list = jieba.cut(data, cut_all=False)
    seg_list = list(seg_list)
    filter_list = []
    wordDict = {}
    jump = False
    for word in seg_list:
        for s in word:
            if not is_chinese(s):
                jump = True
                break
        if not jump:
            filter_list.append(word)
        jump = False
    return filter_list

In [9]:
def analyze(folder, corpus, txts_in_folder):
    # tfidf
    vectorizer = TfidfVectorizer(max_df = 0.9, min_df = 0.2)
    tfidf = vectorizer.fit_transform(corpus)
    words = vectorizer.get_feature_names()
    print("tfidf.shape: ", tfidf.shape)
    for i in range(len(corpus)):
        print('----{0} NEWS----'.format(txts_in_folder[i]))
        for j in range(len(words)):
            if words[j] in imp_words:
                tfidf[i,j] = tfidf[i,j] * 2
            if tfidf[i,j] > 0.2:
                print(words[j], tfidf[i,j], i ,j)
    
    X = tfidf.toarray()
    sklearn_pca = PCA(n_components = 2)
    Y_sklearn = sklearn_pca.fit_transform(X)
    
    return Y_sklearn

In [10]:
def plot_cpa_scatter(Y_sklearn, txts_in_folder):
    pca_data = [
        go.Scatter(
            x = Y_sklearn[:,0],
            y = Y_sklearn[:,1],
            mode = "markers",
            hoverinfo = 'text',
            text = txts_in_folder
        )
    ]

    fig = go.Figure(data = pca_data)
    return py.iplot(fig, filename = 'PCA Scatter Chart')

In [11]:
def plot_kmeans_scatter(Y_sklearn, txts_in_folder):
    kmeans = KMeans(n_clusters = 3)
    X_clustered = kmeans.fit_predict(Y_sklearn)
    
    #Define our own color map
    LABEL_COLOR_MAP = {0:'red', 1: 'green', 2: 'blue'}
    label_color = [LABEL_COLOR_MAP[l] for l in X_clustered]
    
    pca_data = [
        go.Scatter(
            x = Y_sklearn[:,0],
            y = Y_sklearn[:,1],
            mode = "markers",
            hoverinfo = 'text',
            text = txts_in_folder,
            marker = dict(color = label_color)
        )
    ]

    fig = go.Figure(data = pca_data)
    return py.iplot(fig, filename = 'K-means Scatter Chart')

###  分析中電

In [12]:
# get corpus from folder 中電
txts_in_folder0 = get_txts(folders[0])
corpus0 = []

for file_name in txts_in_folder0:
    filter_list = textMining(folders[0], file_name)
    join_list = " ".join(filter_list)
    corpus0.append(join_list)

Y_sklearn0 = analyze(folders[0], corpus0, txts_in_folder0)

tfidf.shape:  (16, 100)
----20170303_1611_投保中心受理中電求償登記.txt NEWS----
利益 0.4729411124893359 0 11
市場 0.7459663196865982 0 29
指出 0.32552613792342705 0 35
符合 0.20069096455421243 0 67
資訊 0.21708295112169412 0 86
----20170327_1611_中電董事會決議出售台南樹谷廠土地.txt NEWS----
中國 0.27218801583609625 1 0
億元 0.47511160354472964 1 6
出售 0.34513328448895936 1 10
利益 0.34513328448895936 1 11
去年 0.2092592225249102 1 13
通過 0.3168367052415047 1 89
電器 0.29291228837888134 1 95
----20170412_1611_中電參加台灣科技展.txt NEWS----
中國 0.23718641370856203 2 0
今日 0.20050093130891167 2 4
參加 0.2552456800801645 2 14
展出 0.27609357310103627 2 26
智慧 0.2858743140627039 2 43
科技展 0.27609357310103627 2 65
系統 0.368124764134715 2 69
路燈 0.368124764134715 2 88
電力 0.20050093130891167 2 94
電器 0.2552456800801645 2 95
----20170628_1611_現任董座周麗真續掌中電經營權.txt NEWS----
今年 0.3267623479426302 3 3
公司 0.3042508987714461 3 9
市場 0.5998657053286761 3 29
科技 0.22383152793961214 3 64
股東 0.3042508987714461 3 74
----20170628_1611_電纜股市場派遭解任.txt NEWS----
今天 0.276280363427426

In [13]:
#plot_cpa_scatter(Y_sklearn0, txts_in_folder0)

In [14]:
plot_kmeans_scatter(Y_sklearn0, txts_in_folder0)

### 分析金雨

In [15]:
# get corpus from folder 金雨
txts_in_folder1 = get_txts(folders[1])
corpus1 = []

for file_name in txts_in_folder1:
    filter_list = textMining(folders[1], file_name)
    join_list = " ".join(filter_list)
    corpus1.append(join_list)

Y_sklearn1 = analyze(folders[1], corpus1, txts_in_folder1)

tfidf.shape:  (8, 188)
----20180223_4503_金雨世代交替.txt NEWS----
卓政 0.22552131105739137 0 36
接任 0.2384611510870755 0 85
業務 0.2384611510870755 0 111
自動 0.2794753710717904 0 158
董事長 0.22227007050236802 0 164
販賣機 0.23955031806153457 0 177
----20180412_4503_借殼上市入主金雨.txt NEWS----
上市 0.26281115208753564 1 0
上櫃 0.24364879813968074 1 2
借殼 0.26281115208753564 1 20
兄弟 0.5256223041750713 1 24
入主 0.4872975962793615 1 25
持股 0.21024892167002848 1 84
----20180412_4503_炒作金雨企業股票遭訴.txt NEWS----
上櫃 0.4502245863998873 2 2
交易 0.22200393006575206 2 11
兄弟 0.22200393006575206 2 24
卓燦然 0.3445950316321042 2 38
興亞 0.27750491258219007 2 160
----20180413_4503_金雨前董座涉炒股.txt NEWS----
兄弟 0.30751066753863343 3 24
入主 0.5345421320056144 3 25
卓燦然 0.417653944929095 3 38
董事會 0.30751066753863343 3 163
起訴 0.23865939710234 3 178
----20180530_4503_受理金雨股價操縱案.txt NEWS----
中心 0.29183570161946526 4 7
交易 0.2218997055946689 4 11
交易法 0.2218997055946689 4 12
台北 0.25715041079041717 4 49
投保 0.3214380134880215 4 80
投資人 0.243196418016221 4 82


/anaconda3/lib/python3.6/site-packages/scipy/sparse/compressed.py:746: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [16]:
plot_kmeans_scatter(Y_sklearn1, txts_in_folder1)

### 分析大同

In [17]:
# get corpus from folder 大同
txts_in_folder2 = get_txts(folders[2])
corpus2 = []

for file_name in txts_in_folder2:
    filter_list = textMining(folders[2], file_name)
    join_list = " ".join(filter_list)
    corpus2.append(join_list)

Y_sklearn2 = analyze(folders[2], corpus2, txts_in_folder2)

tfidf.shape:  (11, 49)
----237120180222.txt NEWS----
----237120180313.txt NEWS----
大同 0.46160774142622 1 10
掩埋 0.7581897554226065 1 15
未來 0.27570536560822057 1 19
發電 0.2067790242061654 1 23
----237120180324.txt NEWS----
中資 0.21922294474672357 2 0
公司 0.33628109554194857 2 3
大同 0.7340808052110163 2 10
經營權 0.6576688342401706 2 27
----237120180325.txt NEWS----
完成 0.5773502691896258 3 11
發電 0.5773502691896258 3 23
綠能 0.5773502691896258 3 28
----237120180326.txt NEWS----
公司 0.3239750590763594 4 3
台北市 0.5280014801363824 4 7
完成 0.4400012334469854 4 11
本案 0.2640007400681912 4 20
發電 0.3520009867575883 4 23
開發 0.2640007400681912 4 44
----237120180328.txt NEWS----
大同 0.6048448391794882 5 10
市場 0.2690372936064086 5 12
經營權 0.20069826073419245 5 27
股東 0.25087282591774057 5 29
董事會 0.20069826073419245 5 31
蔚山 0.6020947822025774 5 32
----237120180330.txt NEWS----
大同 0.38954413445355524 6 10
指出 0.31285280540404187 6 14
股東 0.5816594825631906 6 29
董事會 0.38777298837546037 6 31
----237120180621.txt NEWS----


/anaconda3/lib/python3.6/site-packages/scipy/sparse/compressed.py:746: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [18]:
plot_kmeans_scatter(Y_sklearn2, txts_in_folder2)

### 分析中興電 

In [19]:
# get corpus from folder 中興電
txts_in_folder3 = get_txts(folders[3])
corpus3 = []

for file_name in txts_in_folder3:
    filter_list = textMining(folders[3], file_name)
    join_list = " ".join(filter_list)
    corpus3.append(join_list)

Y_sklearn3 = analyze(folders[3], corpus3, txts_in_folder3)

tfidf.shape:  (7, 149)
----201803291513風電遴選題材發酵.txt NEWS----
上緯 0.29110420070934484 0 0
合作 0.27941102030537357 0 41
建置 0.26150852282160103 0 72
簽署 0.24258683392445404 0 106
遴選 0.24258683392445404 0 130
風電 0.3922627842324015 0 148
----201803311513中興電 配息率達86.21％.txt NEWS----
中興電 0.40273626274914875 1 4
今年 0.23193540281694808 1 11
億元 0.5291785562472412 1 16
去年 0.2713434697135612 1 34
工商 0.46387080563389615 1 67
時報 0.46387080563389615 1 86
----201804051513風電開發商 集結台灣打前哨戰.txt NEWS----
台灣 0.3452965903156897 2 39
日本 0.20717795418941382 2 84
發展 0.20717795418941382 2 101
開發 0.3556733341498701 2 134
離岸 0.20717795418941382 2 137
離岸風電 0.40404189863107576 2 138
----201804091513離岸風電題材續航 華城、大亞、中興電股價齊創波段新高.txt NEWS----
受惠 0.21864004060741668 3 36
大亞 0.24338416804679144 3 60
建置 0.21864004060741668 3 72
成為 0.21864004060741668 3 76
經濟部 0.2803293881204874 3 110
設備 0.21864004060741668 3 124
開發 0.2502339354453587 3 134
離岸風電 0.28426363371021274 3 138
----201806261513外商啟動離岸風電在地佈局 吹響與台廠簽約號角.txt NEWS----
供應商 0.2

In [20]:
plot_kmeans_scatter(Y_sklearn3, txts_in_folder3)

### 分析聲寶

In [21]:
# get corpus from folder 聲寶
txts_in_folder4 = get_txts(folders[4])
corpus4 = []

for file_name in txts_in_folder4:
    filter_list = textMining(folders[4], file_name)
    join_list = " ".join(filter_list)
    corpus4.append(join_list)

Y_sklearn4 = analyze(folders[4], corpus4, txts_in_folder4)

tfidf.shape:  (7, 145)
----160420180129.txt NEWS----
台南 0.5786504812970421 0 32
投資 0.41332177235503015 0 66
----160420180203.txt NEWS----
今年 0.5790157028220855 1 7
去年 0.42110232932515307 1 28
增加 0.2425124018196967 1 46
聲寶 0.2435364955743131 1 113
----160420180321.txt NEWS----
世代 0.23558695390828704 2 1
今年 0.23311023348534907 2 7
平台 0.3356198640712763 2 59
成長 0.46622046697069813 2 65
整合 0.26849589125702106 2 76
----160420180326.txt NEWS----
健康 0.2244429229983596 3 14
台灣 0.3331250392879337 3 33
商機 0.2244429229983596 3 39
熟齡 0.33666438449753944 3 94
產業 0.2244429229983596 3 102
發展 0.2244429229983596 3 105
開創 0.2244429229983596 3 133
----160420180518.txt NEWS----
公司 0.29799931849990624 4 20
合作 0.22882333797855603 4 34
提供 0.29799931849990624 4 70
消費者 0.22882333797855603 4 93
簽約 0.26770263497696284 4 108
開始 0.4015539524654443 4 134
顧客 0.26770263497696284 4 143
----160420180621.txt NEWS----
住宅 0.2118761520064091 5 10
出售 0.2141272672189927 5 24
去年 0.2118761520064091 5 28
土地 0.2141272672189927 5

In [22]:
plot_kmeans_scatter(Y_sklearn4, txts_in_folder4)

### 分析東元 

In [23]:
# get corpus from folder 東元
txts_in_folder5 = get_txts(folders[5])
corpus5 = []

for file_name in txts_in_folder5:
    filter_list = textMining(folders[5], file_name)
    join_list = " ".join(filter_list)
    corpus5.append(join_list)

Y_sklearn5 = analyze(folders[5], corpus5, txts_in_folder5)

tfidf.shape:  (37, 105)
----20180110.txt NEWS----
去年 0.4643988080864487 0 12
效率 0.32431652605851097 0 49
營收 0.5952960193562552 0 60
發電 0.2976480096781276 0 65
----20180118.txt NEWS----
太陽能 0.7184257532650448 1 25
建置 0.41052900186573993 1 33
綠能 0.20526450093286996 1 75
----20180125.txt NEWS----
工業 0.3056613828689828 2 29
市場 0.22152048069189506 2 30
希望 0.3302976252035096 2 32
推出 0.412872031504387 2 43
提供 0.22104925182059398 2 45
產業 0.2750398406350506 2 63
製造 0.22104925182059398 2 81
透過 0.26991859903720866 2 88
需求 0.2292460371517371 2 99
領域 0.2292460371517371 2 102
----20180220.txt NEWS----
主要 0.5082365763095416 3 1
布局 0.5287795155334365 3 31
相關 0.4718423925076823 3 68
領域 0.48933890412065734 3 102
----20180227.txt NEWS----
企業 0.5832937117812887 4 4
東元電機 0.3544001880564301 4 57
相關 0.3328276279401698 4 68
記者 0.40640882660575645 4 84
----20180301.txt NEWS----
公司 0.22731650991884458 5 7
可以 0.2109390081285087 5 14
太陽能 0.5978217487644428 5 25
投資 0.22880452325651912 5 40
政府 0.20309569969052427 5

In [24]:
plot_kmeans_scatter(Y_sklearn5, txts_in_folder5)